# Riesgo de Credito Analisis

In [1]:
import empiricaldist
#import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as ss
import warnings

#Quitar advertencias en los codigos
warnings.filterwarnings('ignore')

# Ver Datasets completos
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

pd.options.display.float_format = '{:.0f}'.format

In [27]:
df_detalle_tasa = pd.read_csv('data\Detalle_Tasa.csv', encoding='latin1')
df_entidades = pd.read_csv('data\Entidades.csv', encoding= 'unicode_escape')
df_plazos = pd.read_csv('data\Plazos.csv', encoding= 'unicode_escape')
df_t_credito = pd.read_csv('data\Tipo_Credito.csv', encoding= 'unicode_escape')
df_t_entidad = pd.read_csv('data\Tipo_entidad.csv', encoding= 'unicode_escape')

In [44]:
df_detalle_tasa.columns

Index(['Nombre_Tipo_Entidad', 'Id_entidad_financiera', 'Fecha_Corte', 'Unicap',
       'Descrip_Unicap', 'Renglon', '(1)Tipo_de_persona', '(2)Sexo',
       '(3)TamaÃ±o_de_empresa', 'cod_sub_tipo', '(5)Tipo_de_garantÃ­a',
       '(6) Producto de crÃ©dito', '(7) Plazo de crÃ©dito',
       '(8)Tasa_efectiva_promedio_ponderada',
       '(9)Margen_adicional_a_la_variaciÃ³n_para_crÃ©ditos_en_UVR',
       '(10)Montos_desembolsados', '(11)NÃºmero_de_crÃ©ditos_desembolsados'],
      dtype='object')

In [29]:
df_detalle_tasa.head(2)

,Nombre_Tipo_Entidad,Id_entidad_financiera,Fecha_Corte,Unicap,Descrip_Unicap,Renglon,(1)Tipo_de_persona,(2)Sexo,(3)TamaÃ±o_de_empresa,cod_sub_tipo,(5)Tipo_de_garantÃ­a,(6) Producto de crÃ©dito,(7) Plazo de crÃ©dito,(8)Tasa_efectiva_promedio_ponderada,(9)Margen_adicional_a_la_variaciÃ³n_para_crÃ©ditos_en_UVR,(10)Montos_desembolsados,(11)NÃºmero_de_crÃ©ditos_desembolsados
0,BC-ESTABLECIMIENTO BANCARIO,49,16/09/2022,1,TASAS DE INTERESES ACTIVAS,125,Juridica,No aplica,Mediana empresa,1,Sin Garantia,Tarjeta de crÃ©dito empresarial,Consumos a un mes,0,NaN,2468088,2
1,BC-ESTABLECIMIENTO BANCARIO,49,22/07/2022,1,TASAS DE INTERESES ACTIVAS,137,Juridica,No aplica,Gran empresa,1,GarantÃ­a idÃ³nea o no idÃ³nea,Sobregiro en cuenta corriente persona jurÃ­dica,Hasta 30 dÃ­as,32,NaN,708926352,3


In [19]:
df_entidades.head(1)

,Id_entidad_financiera,Entidad_financiera
0,7,Bancolombia


In [20]:
df_plazos.head(1)

,Cod_Plazo,Plazo_credito
0,1,A Más de 20 años


In [25]:
df_t_credito.head(2)

,tipo_credito,descripcion_tipo,cod_sub_tipo,sub_tipo_credito
0,2,Consumo,1,Consumo
1,1,Comercial,1,Comercial ordinario


In [39]:
df_sub_tipo = df_t_credito.groupby(['cod_sub_tipo','sub_tipo_credito'] , as_index=False).sum()
df_sub_tipo = df_sub_tipo.reindex(columns=['cod_sub_tipo','sub_tipo_credito'])
df_sub_tipo = df_sub_tipo.drop(df_sub_tipo.index[[0]])
df_sub_tipo

,cod_sub_tipo,sub_tipo_credito
1,1,Consumo
2,2,Comercial preferencial o corporativo
3,4,Vivienda
4,5,Microcrédito
5,6,Comercial tesorería
6,7,Comercial especial


In [26]:
df_t_entidad.head(5)

,Tipo_Entidad,Nombre_Tipo_Entidad
0,1,BC-ESTABLECIMIENTO BANCARIO
1,4,CF-COMPANIA DE FINANCIAMIENTO
2,22,INSTITUCION OFICIAL ESPECIAL
3,32,COOPERATIVAS DE CARACTER FINANCIERO


In [40]:
df_left_tasa_entidad = df_detalle_tasa.merge(df_entidades, left_on=['Id_entidad_financiera'], right_on=['Id_entidad_financiera'], how='left')
df_left_tasa_entidad_sub = df_left_tasa_entidad.merge(df_sub_tipo, left_on=['cod_sub_tipo'], right_on=['cod_sub_tipo'], how='left')

In [51]:
tasa_maxima = df_left_tasa_entidad_sub.groupby(['Entidad_financiera','Nombre_Tipo_Entidad','sub_tipo_credito']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'tasa': x['(8)Tasa_efectiva_promedio_ponderada'].unique().max()
                                })).reset_index()

In [115]:
tasa_maxima.head(20).sort_values('tasa',ascending=False)

,Entidad_financiera,Nombre_Tipo_Entidad,sub_tipo_credito,tasa
16,Banco Davivienda,BC-ESTABLECIMIENTO BANCARIO,Consumo,45
4,BBVA Colombia,BC-ESTABLECIMIENTO BANCARIO,Consumo,37
9,Bancamia S.A.,BC-ESTABLECIMIENTO BANCARIO,Consumo,37
1,AV Villas,BC-ESTABLECIMIENTO BANCARIO,Consumo,37
18,Banco Falabella S.A.,BC-ESTABLECIMIENTO BANCARIO,Consumo,37
7,Banagrario,BC-ESTABLECIMIENTO BANCARIO,Consumo,37
13,Banco Caja Social S.A.,BC-ESTABLECIMIENTO BANCARIO,Consumo,36
12,Banco Caja Social S.A.,BC-ESTABLECIMIENTO BANCARIO,Comercial preferencial o corporativo,29
3,BBVA Colombia,BC-ESTABLECIMIENTO BANCARIO,Comercial preferencial o corporativo,25
15,Banco Davivienda,BC-ESTABLECIMIENTO BANCARIO,Comercial preferencial o corporativo,23


In [98]:
df_left_tasa_entidad_sub['TIME'] = pd.to_datetime(df_left_tasa_entidad_sub['Fecha_Corte'], format="%d/%m/%Y")
df_left_tasa_entidad_sub['year'] = df_left_tasa_entidad_sub['TIME'].dt.year
df_left_tasa_entidad_sub['month'] = df_left_tasa_entidad_sub['TIME'].dt.month

In [99]:
tasa_promedio = df_left_tasa_entidad_sub.groupby(['year','month','Nombre_Tipo_Entidad','sub_tipo_credito']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'tasa_promedio': np.median(x['(8)Tasa_efectiva_promedio_ponderada'].unique()),
                                    'Monto_total_desembolsado': x['(10)Montos_desembolsados'].sum(),
                                    'cantidad_desembolsos': x['(11)NÃºmero_de_crÃ©ditos_desembolsados'].sum()
                                })).reset_index()

tasa_promedio

,year,month,Nombre_Tipo_Entidad,sub_tipo_credito,tasa_promedio,Monto_total_desembolsado,cantidad_desembolsos
0,2022,7,BC-ESTABLECIMIENTO BANCARIO,Comercial especial,12,93649734819,750
1,2022,7,BC-ESTABLECIMIENTO BANCARIO,Comercial preferencial o corporativo,13,8216044679410,25576
2,2022,7,BC-ESTABLECIMIENTO BANCARIO,Consumo,20,49207140648987,46240701
3,2022,7,BC-ESTABLECIMIENTO BANCARIO,Vivienda,13,3263913564903,27583
4,2022,7,CF-COMPANIA DE FINANCIAMIENTO,Comercial preferencial o corporativo,15,10000000,2
5,2022,7,CF-COMPANIA DE FINANCIAMIENTO,Consumo,21,2030890990010,2153116
6,2022,7,CF-COMPANIA DE FINANCIAMIENTO,Vivienda,14,30274970081,441
7,2022,7,COOPERATIVAS DE CARACTER FINANCIERO,Consumo,21,412685936375,68158
8,2022,7,COOPERATIVAS DE CARACTER FINANCIERO,Vivienda,12,14429843794,283
9,2022,7,INSTITUCION OFICIAL ESPECIAL,Comercial especial,12,356194374039,63


In [100]:
octubre_2022 = df_left_tasa_entidad_sub[df_left_tasa_entidad_sub['Fecha_Corte'].str.contains("10/2022")]
octubre_2022['mes'] = 'octubre_2022'
octubre_2022.head(5)

,Nombre_Tipo_Entidad,Id_entidad_financiera,Fecha_Corte,Unicap,Descrip_Unicap,Renglon,(1)Tipo_de_persona,(2)Sexo,(3)TamaÃ±o_de_empresa,cod_sub_tipo,(5)Tipo_de_garantÃ­a,(6) Producto de crÃ©dito,(7) Plazo de crÃ©dito,(8)Tasa_efectiva_promedio_ponderada,(9)Margen_adicional_a_la_variaciÃ³n_para_crÃ©ditos_en_UVR,(10)Montos_desembolsados,(11)NÃºmero_de_crÃ©ditos_desembolsados,Entidad_financiera,sub_tipo_credito,TIME,year,month,mes
5,BC-ESTABLECIMIENTO BANCARIO,49,21/10/2022,1,TASAS DE INTERESES ACTIVAS,47,Natural,Femenino,No aplica,1,GarantÃ­a idÃ³nea o no idÃ³nea,Sobregiro en cuenta corriente persona natural,Entre 31 dÃ­as y 1 aÃ±o,37,NaN,5254715,3,AV Villas,Consumo,2022-10-21,2022,10,octubre_2022
9,BC-ESTABLECIMIENTO BANCARIO,49,14/10/2022,1,TASAS DE INTERESES ACTIVAS,146,Juridica,No aplica,Gran empresa,2,GarantÃ­a idÃ³nea o no idÃ³nea,Corporativo,MÃ¡s de 3 aÃ±os y hasta 5 aÃ±os,16,NaN,288585914,3,AV Villas,Comercial preferencial o corporativo,2022-10-14,2022,10,octubre_2022
10,BC-ESTABLECIMIENTO BANCARIO,49,14/10/2022,1,TASAS DE INTERESES ACTIVAS,96,Natural,Masculino,No aplica,1,GarantÃ­a idÃ³nea o no idÃ³nea,Tarjeta de crÃ©dito para ingresos superiores a...,Consumos a un mes,0,NaN,58599949,372,AV Villas,Consumo,2022-10-14,2022,10,octubre_2022
13,BC-ESTABLECIMIENTO BANCARIO,49,21/10/2022,1,TASAS DE INTERESES ACTIVAS,101,Natural,Masculino,No aplica,1,GarantÃ­a idÃ³nea o no idÃ³nea,Sobregiro en cuenta corriente persona natural,Hasta 30 dÃ­as,37,NaN,6934157,6,AV Villas,Consumo,2022-10-21,2022,10,octubre_2022
15,BC-ESTABLECIMIENTO BANCARIO,49,7/10/2022,1,TASAS DE INTERESES ACTIVAS,58,Natural,Masculino,No aplica,1,Sin Garantia,Consumo - CrÃ©dito rotativo,MÃ¡s de 1 aÃ±o y hasta 3 aÃ±os,26,NaN,72646030,118,AV Villas,Consumo,2022-10-07,2022,10,octubre_2022


In [84]:
monto_desembolsos_oct = octubre_2022.groupby(['mes','(3)TamaÃ±o_de_empresa']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'Monto_total_desembolsado_promedio': np.median(x['(10)Montos_desembolsados']),
                                    'Monto_total_desembolsado_total': x['(10)Montos_desembolsados'].sum(),
                                    'cantidad_desembolsos': x['(11)NÃºmero_de_crÃ©ditos_desembolsados'].sum()
                                })).reset_index()
monto_desembolsos_oct

,mes,(3)TamaÃ±o_de_empresa,Monto_total_desembolsado_promedio,Monto_total_desembolsado_total,cantidad_desembolsos
0,octubre_2022,Gran empresa,1700190135,20724230608485,984640
1,octubre_2022,Mediana empresa,650000000,5275373877000,254205
2,octubre_2022,Microempresa,116590000,3577480561124,873419
3,octubre_2022,No aplica,171711582,20717168466580,40534531
4,octubre_2022,PequeÃ±a empresa,200000000,2770392099929,364272


In [89]:
monto_tasa = octubre_2022.groupby(['mes','(1)Tipo_de_persona','(6) Producto de crÃ©dito']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'tasa_promedio': np.median(x['(8)Tasa_efectiva_promedio_ponderada']),
                                    'Monto_total_desembolsado_promedio': np.median(x['(10)Montos_desembolsados']),
                                    'Monto_total_desembolsado_total': x['(10)Montos_desembolsados'].sum(),
                                    'cantidad_desembolsos': x['(11)NÃºmero_de_crÃ©ditos_desembolsados'].sum()
                                })).reset_index()
monto_tasa[monto_tasa['(1)Tipo_de_persona'].str.contains("Natural")]

,mes,(1)Tipo_de_persona,(6) Producto de crÃ©dito,tasa_promedio,Monto_total_desembolsado_promedio,Monto_total_desembolsado_total,cantidad_desembolsos
25,octubre_2022,Natural,AdquisiciÃ³n de vivienda no vis (colocaciÃ³n e...,15,664145000,991104511283,5980
26,octubre_2022,Natural,AdquisiciÃ³n de vivienda no vis (colocaciÃ³n e...,19,325501584,66345971474,392
27,octubre_2022,Natural,AdquisiciÃ³n de vivienda vis (colocaciÃ³n en p...,15,187920000,642013716965,11096
28,octubre_2022,Natural,AdquisiciÃ³n de vivienda vis (colocaciÃ³n en uvr),19,188881564,141652232932,2106
29,octubre_2022,Natural,AdquisiciÃ³n leasing habitacional no vis (colo...,15,805198295,348471898348,1269
30,octubre_2022,Natural,AdquisiciÃ³n leasing habitacional no vis (colo...,19,461697600,9588375560,33
31,octubre_2022,Natural,AdquisiciÃ³n leasing habitacional vis (colocac...,15,183300000,4746571116,41
32,octubre_2022,Natural,Comerciales con recursos provenientes de redes...,18,200000000,137370583840,2490
33,octubre_2022,Natural,Comerciales especiales establecidos mediante l...,18,15636320,49464720,9
34,octubre_2022,Natural,ConstrucciÃ³n - financiaciÃ³n de proyectos viales,22,104500000,209000000,2


In [101]:
monto_plazos_tasa = octubre_2022.groupby(['mes','(7) Plazo de crÃ©dito']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'tasa_promedio': np.median(x['(8)Tasa_efectiva_promedio_ponderada'])
                                })).reset_index()
monto_plazos_tasa

,mes,(7) Plazo de crÃ©dito,tasa_promedio
0,octubre_2022,A mÃ¡s de 20 aÃ±os,16
1,octubre_2022,A mÃ¡s de 7 aÃ±os,18
2,octubre_2022,Avances en efectivo,36
3,octubre_2022,Consumos a mÃ¡s de 36 meses,35
4,octubre_2022,Consumos a un mes,3
5,octubre_2022,Consumos entre 13 y 18 meses,36
6,octubre_2022,Consumos entre 19 y 24 meses,36
7,octubre_2022,Consumos entre 2 y 6 meses,36
8,octubre_2022,Consumos entre 25 y 36 meses,36
9,octubre_2022,Consumos entre 7 y 12 meses,35


In [94]:
monto_plazos_tasa = octubre_2022.groupby(['mes','(2)Sexo','(5)Tipo_de_garantÃ­a']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'Monto_total_desembolsado_total': x['(10)Montos_desembolsados'].sum(),
                                    'cantidad_desembolsos': x['(11)NÃºmero_de_crÃ©ditos_desembolsados'].sum()
                                })).reset_index()
monto_plazos_tasa

,mes,(2)Sexo,(5)Tipo_de_garantÃ­a,Monto_total_desembolsado_total,cantidad_desembolsos
0,octubre_2022,Femenino,GarantÃ­a fondo nacional de garantÃ­as (FNG) ...,156492190649,17035
1,octubre_2022,Femenino,GarantÃ­a del fondo agropecuario de garantÃ­as...,100730459586,8561
2,octubre_2022,Femenino,GarantÃ­a idÃ³nea o no idÃ³nea,2549221980805,1781947
3,octubre_2022,Femenino,Sin Garantia,6396923049201,14555444
4,octubre_2022,Masculino,GarantÃ­a fondo nacional de garantÃ­as (FNG) ...,181036389467,14256
5,octubre_2022,Masculino,GarantÃ­a del fondo agropecuario de garantÃ­as...,187200903670,13310
6,octubre_2022,Masculino,GarantÃ­a idÃ³nea o no idÃ³nea,3741443321422,2529257
7,octubre_2022,Masculino,Sin Garantia,9753203039846,22229083
8,octubre_2022,No aplica,GarantÃ­a fondo nacional de garantÃ­as (FNG) ...,311927696082,2410
9,octubre_2022,No aplica,GarantÃ­a del fondo agropecuario de garantÃ­as...,101530635105,172


In [106]:
df_left_tasa_entidad_sub.columns

Index(['Nombre_Tipo_Entidad', 'Id_entidad_financiera', 'Fecha_Corte', 'Unicap',
       'Descrip_Unicap', 'Renglon', '(1)Tipo_de_persona', '(2)Sexo',
       '(3)TamaÃ±o_de_empresa', 'cod_sub_tipo', '(5)Tipo_de_garantÃ­a',
       '(6) Producto de crÃ©dito', '(7) Plazo de crÃ©dito',
       '(8)Tasa_efectiva_promedio_ponderada',
       '(9)Margen_adicional_a_la_variaciÃ³n_para_crÃ©ditos_en_UVR',
       '(10)Montos_desembolsados', '(11)NÃºmero_de_crÃ©ditos_desembolsados',
       'Entidad_financiera', 'sub_tipo_credito', 'TIME', 'year', 'month',
       'Fecha'],
      dtype='object')

In [111]:
df_left_tasa_entidad_sub
df_left_tasa_entidad_sub['Fecha'] = df_left_tasa_entidad_sub['year'].astype(str) + df_left_tasa_entidad_sub['month'].astype(str)
df_left_tasa_entidad_sub_reporte = df_left_tasa_entidad_sub.groupby(['Fecha','Nombre_Tipo_Entidad','Id_entidad_financiera','Entidad_financiera','(1)Tipo_de_persona','(3)TamaÃ±o_de_empresa','(7) Plazo de crÃ©dito']\
        ).apply(lambda x: pd.Series( \
                                    { \
                                    'tasa_promedio_mensual': np.median(x['(8)Tasa_efectiva_promedio_ponderada']),
                                    'margen_maximo': x['(9)Margen_adicional_a_la_variaciÃ³n_para_crÃ©ditos_en_UVR'].unique().max(),
                                    'tasa_maximo': x['(8)Tasa_efectiva_promedio_ponderada'].unique().max(),
                                    'tasa_minima': x['(8)Tasa_efectiva_promedio_ponderada'].unique().min(),
                                    'Valor_desembolsos': x['(10)Montos_desembolsados'].sum(),
                                    'monto_maximo': x['(10)Montos_desembolsados'].unique().max(),
                                    'monto_minima': x['(10)Montos_desembolsados'].unique().min(),
                                    'cantidad_desembolsos': x['(11)NÃºmero_de_crÃ©ditos_desembolsados'].sum()
                                })).reset_index()
df_left_tasa_entidad_sub_reporte

,Fecha,Nombre_Tipo_Entidad,Id_entidad_financiera,Entidad_financiera,(1)Tipo_de_persona,(3)TamaÃ±o_de_empresa,(7) Plazo de crÃ©dito,tasa_promedio_mensual,margen_maximo,tasa_maximo,tasa_minima,Valor_desembolsos,monto_maximo,monto_minima,cantidad_desembolsos
0,202210,BC-ESTABLECIMIENTO BANCARIO,1,Banco de Bogota,Juridica,Gran empresa,A mÃ¡s de 7 aÃ±os,16,NaN,17,14,28829782963,15000000000,519800000,7
1,202210,BC-ESTABLECIMIENTO BANCARIO,1,Banco de Bogota,Juridica,Gran empresa,Avances en efectivo,37,NaN,37,37,6700000,4500000,1000000,12
2,202210,BC-ESTABLECIMIENTO BANCARIO,1,Banco de Bogota,Juridica,Gran empresa,Consumos a un mes,0,NaN,0,0,43294325132,12254349550,9638418471,49947
3,202210,BC-ESTABLECIMIENTO BANCARIO,1,Banco de Bogota,Juridica,Gran empresa,Consumos entre 13 y 18 meses,37,NaN,37,35,197245,96000,35045,3
4,202210,BC-ESTABLECIMIENTO BANCARIO,1,Banco de Bogota,Juridica,Gran empresa,Consumos entre 19 y 24 meses,37,NaN,37,37,25870210,16004313,1086104,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,20229,INSTITUCION OFICIAL ESPECIAL,8,Giros y Finanzas C.F.,Natural,No aplica,MÃ¡s de 5 aÃ±os y hasta 7 aÃ±os,10,NaN,10,10,11000000,11000000,11000000,1
7346,20229,INSTITUCION OFICIAL ESPECIAL,8,Giros y Finanzas C.F.,Natural,No aplica,MÃ¡s de 7 aÃ±os y hasta 15 aÃ±os,9,NaN,11,6,3858846989,442169631,6334090,47
7347,20229,INSTITUCION OFICIAL ESPECIAL,8,Giros y Finanzas C.F.,Natural,No aplica,Vivienda a 5 aÃ±os,9,NaN,9,8,102556106,67556106,35000000,3
7348,20229,INSTITUCION OFICIAL ESPECIAL,11,Caja de Vivienda Militar y de Policia,Natural,No aplica,MÃ¡s de 5 aÃ±os y hasta 7 aÃ±os,9,NaN,10,9,9763750000,2584000000,90950000,17


In [112]:
df_left_tasa_entidad_sub_reporte.to_csv('data/reporte_consolidado.csv')